In [ ]:
import os 

from PIL import Image

In [ ]:
finetune_path = "sdv1-5-var-aspect-3"

gen_folder = os.path.join(finetune_path, "generations")
video_folder = os.path.join(finetune_path, "videos")
os.makedirs(video_folder, exist_ok=True)

step_names = sorted([f for f in os.listdir(gen_folder) if not f.endswith(".ipynb_checkpoints")],  key=lambda x: int(x))

In [ ]:
import imageio
import numpy as np
from PIL import ImageDraw
import torchvision
import torch

prompt_num = 6
resolutions = ["256x256", "512x512", "1024x512", "512x1024"]


to_tensor = torchvision.transforms.ToTensor()
to_pil = torchvision.transforms.ToPILImage()
for res in resolutions:
    skip_res = False
    
    all_prompts = []
    for prompt_id in range(prompt_num):
        img_name = f"{prompt_id}_{res}.jpg"
        img_paths = [os.path.join(gen_folder, f, img_name) for f in step_names]
        all_prompts.append(img_paths)    

    # create video
    vid_name = f"{res}.mp4"
    vid_path = os.path.join(video_folder, vid_name)
    
    num_prompts = len(all_prompts[0])
    # write video
    writer = imageio.get_writer(vid_path, fps=4)
    for i in range(num_prompts):
        img_list = []
        for img_paths in all_prompts:
            #try:
                img_path = img_paths[i]
                tens = to_tensor(Image.open(img_path))
                img_list.append(tens)
            #except FileNotFoundError:
            #    print("Could not load ", img_path)
            #    skip_res = True
                
        if skip_res == True:
            print("Skipping:", res)
            break
        grid = torchvision.utils.make_grid(img_list, nrow = 3, padding = 2,
                                           normalize=False, scale_each = False, pad_value = 0.0)
        ndarr = grid.mul(255).add_(0.5).clamp_(0, 255).permute(1, 2, 0).to("cpu", torch.uint8).numpy()
        grid_img = Image.fromarray(ndarr)
        edit_img = ImageDraw.Draw(grid_img)
        step = step_names[i]
        text = f"Step {step}"
        edit_img.text((15, 15), text, (237, 230, 211))
        img_arr = np.array(grid_img)

        writer.append_data(img_arr)
    writer.close()
print("Done")